In [1]:
from simulators import simulator
from spectralcomponents import PowerLaw, GaussianEmissionLine

In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sbi import utils
from sbi import analysis
from sbi import inference
from sbi.inference import SNPE, simulate_for_sbi, prepare_for_sbi

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-24 14:31:06.478302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sbi.neural_nets.embedding_nets import FCEmbedding, CNNEmbedding, PermutationInvariantEmbedding

embedding_net = CNNEmbedding(input_shape=(1000,))



In [4]:
c1 = PowerLaw()
true_params = (1, 1)
prior_min = [0.1, 0.1]
prior_max = [2.0, 2.0]
prior = utils.torchutils.BoxUniform(
    low=torch.as_tensor(prior_min), high=torch.as_tensor(prior_max)
)

def simulate_simple(params):
    return simulator(c1, 1000, (params), pileup='bins')


In [5]:
simulator_wrapper, prior = prepare_for_sbi(simulate_simple, prior)

neural_posterior = utils.posterior_nn(
    model="maf", embedding_net=embedding_net, hidden_features=10, num_transforms=2
)

# setup the inference procedure with the SNPE-C procedure
inference = SNPE(prior=prior, density_estimator=neural_posterior)

In [6]:
alpha, beta = simulate_for_sbi(simulator_wrapper, prior, num_simulations=1000)

Running 1000 simulations.:   8%|▊         | 75/1000 [11:22<3:22:06, 13.11s/it]